In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Loading required packages

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score,make_scorer,confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
import re
import nltk
nltk.download(['punkt','wordnet','stopwords'])
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
import string

## Set seed to get the same results each time

In [ ]:
np.random.seed(0)

## Load the training data

In [ ]:
full_data = pd.read_csv("../input/commonlitreadabilityprize/train.csv")

In [ ]:
full_data.head()

## Separate the text from the target- readability score

In [ ]:
excerpts = full_data["excerpt"]
target = full_data["target"]

In [ ]:
def tokenize(text):
    table = text.maketrans(dict.fromkeys(string.punctuation))
    words = word_tokenize(text.lower().strip().translate(table))
    words = [word for word in words if word not in stopwords.words('english')]
    lemmed = [WordNetLemmatizer().lemmatize(word) for word in words]
    lemmed = [WordNetLemmatizer().lemmatize(word, pos='v') for word in lemmed]
    stemmed = [PorterStemmer().stem(word) for word in lemmed]
    
    return stemmed

These tokenization are basics. They may or maynot be good for perfromance for this particular competition.

In [ ]:
excerpts[5]

In [ ]:
tokenize(excerpts[5])

Whole pipeline from text processing(lemmetaization,stemming) to Modelling.

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestRegressor(random_state=42))
])

splitting the dataset for making validation set

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(excerpts, target, random_state=42)

In [ ]:
import time
time_begin = time.time()
pipeline.fit(X_train, y_train)
print(time.time() - time_begin)

In [ ]:
y_pred = pipeline.predict(X_val)

In [ ]:
mean_absolute_error(y_val,y_pred)

Prediction on Test Data.

In [ ]:
test_data = pd.read_csv("../input/commonlitreadabilityprize/test.csv")

In [ ]:
test_excerpts = test_data["excerpt"]

In [ ]:
y_sub=pipeline.predict(test_excerpts)

submission ready

In [ ]:
x_sub = test_data[["id"]].copy()
x_sub["target"] = y_sub
x_sub.to_csv('submission.csv', index = False)
x_sub